# Notebook: Generating QA Pairs for IU X-ray Fine-Tuning

This notebook prepares the high-quality instruction-tuning dataset used to fine-tune our multimodal radiology agent. It generates a curated set of ~1,500 question-answer pairs based on the **IU-Xray "impression" field**, covering both template-based and GPT-4o-enhanced paraphrases. This dataset is optimized for use in LoRA fine-tuning of a BioGPT QA model.

---

## Background

In order to teach our model how to interpret and respond to clinical radiology summaries, we curate a structured instruction-tuning dataset using the impression field from the IU-Xray corpus. These radiology impressions resemble real-world CXR reports and serve as a natural foundation for medical Q/A generation.

The goal of this notebook is to generate three high-quality QA pairs per impression:

1. **Q1 (Template):** "What is the radiologic impression?"
2. **Q2 (Template):** "Summarize the key thoracic findings."
3. **Q3 (GPT-4o):** A paraphrased question from a small template pool, with a paraphrased answer generated by GPT-4o.

This combination provides both linguistic diversity and task consistency, enhancing the quality and variety of the fine-tuning dataset.

To ensure precision:
- **Normal/abnormal filtering** was used to bias toward informative cases.
- **GPT-4o** was instructed to keep responses under 25 tokens.
- **Temperature** was set to `0.3` for reproducibility and minimal hallucination.
- **Hallucination risk** was mitigated through careful prompt design and filtering rules.

---

## Workflow Overview

### **Step 1 – Impression Filtering**
- Loaded `iu_impr.jsonl` containing raw impressions.
- Applied regex-based heuristics to segment into:
  - `Tier A` – clearly abnormal, clean impressions
  - `Tier B` – mildly corrupted with redactions (e.g., "XXXX")
  - `Tier C` – likely normal cases
- Removed overly short or severely corrupted entries.
- Combined Tiers A and B and saved 520 selected impressions for QA generation.

### **Step 2 – Sampling**
- Randomly sampled 500+ impressions for generation, ensuring sufficient linguistic diversity.
- Saved result to `iu_sampled_for_qa.jsonl` for reproducibility.

### **Step 3 – QA Generation**
- For each impression, generated 3 QA pairs:
  - Q1 and Q2 using fixed templates
  - Q3 using GPT-4o paraphrasing (with hallucination-aware prompt)
- Temperature fixed at 0.3 for deterministic output.
- GPT-4o instructed to rephrase impression within 25 tokens, preserving all findings.
- Saved final dataset as `qa_pairs_lora.jsonl`.

### **Step 4 – Train/Val Split**
- Split final QA dataset 90%/10% into `train.jsonl` and `val.jsonl`
- Each record contains: `uuid`, `question`, and `answer`
- Enables reproducible training and evaluation during LoRA fine-tuning.

---

### **Final Output Directory**
All generated assets are saved to the following directory for downstream use:

```
data/iu_xray/
├── iu_impr.jsonl              # Raw input impressions
├── iu_sampled_for_qa.jsonl    # Filtered & sampled for QA generation
└── qapairs/
    ├── qa_pairs_lora.jsonl    # Final QA dataset (3 × 500+)
    ├── train.jsonl            # 90% training split
    └── val.jsonl              # 10% validation split
```


## Step 0: Mounting Google Drive and Importing Libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/multimodal-xray-agent
!ls

In [3]:
import os
import re
import json
import openai
import random

from tqdm import tqdm
from pathlib import Path
from openai import OpenAI
from getpass import getpass

In [4]:
# Secure OpenAI API key entry
os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")

Enter your OpenAI API key: ··········


In [5]:
from src.qa_utils import (
    extract_impression,
    has_abnormal_keyword,
    normalize_start,
    replace_redactions,
    gpt_paraphrase,
    dump,
)

## Step 1: Load + Clean Raw IU Impressions

In [6]:
# Setting Paths

PROJECT_ROOT = Path("/content/drive/MyDrive/multimodal-xray-agent")
IMPR_JSONL_PATH = PROJECT_ROOT / "data" / "iu_xray" / "iu_impr.jsonl"
SAMPLED_IMPRESSIONS_PATH = PROJECT_ROOT / "data" / "iu_xray" / "iu_sampled_for_qa.jsonl"
QA_DIR = PROJECT_ROOT / "data" / "qapairs"
OUTPUT_JSONL = QA_DIR / "qa_pairs_lora.jsonl"

VAL_PATH = QA_DIR / "val.jsonl"
TRAIN_PATH = QA_DIR / "train.jsonl"

os.makedirs(QA_DIR, exist_ok=True)

In [69]:
# Tier definitions
tier_a, tier_b, tier_c = [], [], []

NEGATION_REGEX = re.compile(
    r"^(1\.\s*)?(no\s+(acute|evidence|active|abnormality|radiographic|abnormal|significant)|"
    r"without|normal|clear|heart size is normal|there is no|negative)\b",
    re.IGNORECASE
)

# Inclusion heuristics
MAX_REDACTED_TOKENS = 3
MIN_WORDS = 10

In [71]:
with open(IMPR_JSONL_PATH, "r") as f:
    records = [json.loads(line) for line in f]

In [72]:
for rec in tqdm(records, desc="Dividing impressions"):
    raw = rec["impression"]
    uuid = rec["uuid"]
    impr = extract_impression(raw)

    # Skip empty or extremely short
    if not impr or len(impr) < 5:
        continue

    # Count and evaluate redactions
    redacted_count = impr.count(REDACTION_TOKEN)
    if redacted_count > MAX_REDACTED_TOKENS:
        continue

    # Replace "XXXX" for Tier B impressions
    impr = replace_redactions(impr)
    word_count = len(impr.split())
    abnormal_flag = has_abnormal_keyword(impr)
    normalized = normalize_start(impr)

    record = {"uuid": uuid, "impression": impr}

    # Tier assignment
    if redacted_count > 0:
        if word_count >= MIN_WORDS or abnormal_flag:
            tier_b.append(record)
    elif NEGATION_REGEX.match(normalized):
        tier_c.append(record)
    else:
        if word_count >= MIN_WORDS or abnormal_flag:
            tier_a.append(record)

print(f"\nTier A (clean abnormal): {len(tier_a)}")
print(f"Tier B (mildly redacted): {len(tier_b)}")
print(f"Tier C (likely normal): {len(tier_c)}")

Dividing impressions: 100%|██████████| 7430/7430 [00:00<00:00, 106404.98it/s]


Tier A (clean abnormal): 1278
Tier B (mildly redacted): 1014
Tier C (likely normal): 4395


In [73]:
# Examples
tier_a[:10]

[{'uuid': 'iu_0005',
  'impression': '1. Ill-defined opacity in the lingula. Increased since the previous study. May represent increased atelectasis or scarring'},
 {'uuid': 'iu_0007',
  'impression': 'Heart size moderately enlarged for technique, aortic ectasia/tortuosity. Right hemidiaphragm eventration. No focal alveolar consolidation, no definite pleural effusion seen. Vascular redistribution without typical findings of pulmonary edema. No pneumothorax'},
 {'uuid': 'iu_0012',
  'impression': '1. Patchy left lower lobe airspace disease, possibly atelectasis or pneumonia. 2. Right mid lung subsegmental atelectasis'},
 {'uuid': 'iu_0013',
  'impression': 'Borderline cardiomegaly without acute disease'},
 {'uuid': 'iu_0014', 'impression': 'No focal infiltrate'},
 {'uuid': 'iu_0023',
  'impression': 'Borderline cardiomegaly without acute disease'},
 {'uuid': 'iu_0028',
  'impression': '1. Mild cardiomegaly and moderate hiatal hernia'},
 {'uuid': 'iu_0035',
  'impression': '1. Moderate r

In [74]:
tier_b[:10]

[{'uuid': 'iu_0006',
  'impression': '1. Probable [REDACTED] residual left pneumothorax. 2. Stable streaky left basilar airspace disease, possibly atelectasis'},
 {'uuid': 'iu_0008',
  'impression': 'Comparison [REDACTED], [REDACTED] Well-expanded and clear lungs. Mediastinal contour within normal limits. No acute cardiopulmonary abnormality identified. Stable chest'},
 {'uuid': 'iu_0010',
  'impression': '1. Right basilar opacities favored to represent atelectasis. 2. Stigmata of [REDACTED] cell disease. 3. Cholelithiasis'},
 {'uuid': 'iu_0018',
  'impression': '1. Left midlung opacity, not well seen on prior exam, may represent focus of airspace disease. 2. Stable left base opacities, [REDACTED] scarring or atelectasis. 2. Postsurgical changes as above'},
 {'uuid': 'iu_0025',
  'impression': '1. No evidence of active tuberculosis. 2. [REDACTED] left upper lobe opacities. The appearance [REDACTED] subsegmental atelectasis or scarring'},
 {'uuid': 'iu_0031',
  'impression': 'Numerous b

In [75]:
tier_c[:10]

[{'uuid': 'iu_0000',
  'impression': 'Clear lungs with no suspicious pulmonary nodules or masses'},
 {'uuid': 'iu_0003', 'impression': 'No acute cardiopulmonary process'},
 {'uuid': 'iu_0004', 'impression': '1. No acute cardiopulmonary findings'},
 {'uuid': 'iu_0009', 'impression': 'No acute cardiopulmonary findings'},
 {'uuid': 'iu_0015', 'impression': 'No acute cardiopulmonary finding'},
 {'uuid': 'iu_0017', 'impression': 'No acute pulmonary disease'},
 {'uuid': 'iu_0019', 'impression': 'No acute cardiopulmonary abnormality'},
 {'uuid': 'iu_0020', 'impression': 'No acute cardiopulmonary finding'},
 {'uuid': 'iu_0021', 'impression': 'No acute cardiopulmonary abnormalities'},
 {'uuid': 'iu_0022',
  'impression': 'No acute radiographic cardiopulmonary process'}]

## Step 2: Sampling Impressions for QA Generation

In [76]:
# Set reproducibility
random.seed(42)

In [77]:
# Sample sizes
N_A, N_B, N_C = 320, 100, 100

In [78]:
# Shuffle and sample
sampled_a = random.sample(tier_a, N_A)
sampled_b = random.sample(tier_b, N_B)
sampled_c = random.sample(tier_c, N_C)

In [79]:
# Concatenate
sampled_qa_pool = sampled_a + sampled_b + sampled_c
random.shuffle(sampled_qa_pool)

In [109]:
sampled_qa_pool[0:10]

[{'uuid': 'iu_5728',
  'impression': 'Right lower lobe pneumonia. Consider followup radiograph to document resolution'},
 {'uuid': 'iu_3979',
  'impression': 'Small left retrocardiac opacity, may represent minimal atelectasis or small focus of airspace disease'},
 {'uuid': 'iu_4920',
  'impression': '1.5 cm nodule in the left midlung zone. May contain calcium although this cannot be stated with certainty. Comparison to old films, if available would be useful to determine chronicity and stability. If old films are not available, [REDACTED] scan could be [REDACTED] to evaluate for the presence of calcium'},
 {'uuid': 'iu_2857',
  'impression': "Exam is technically limited by patient's body habitus. Lung volumes are decreased. No evidence of acute airspace disease, large pleural effusion or pneumothorax. Heart size is stable"},
 {'uuid': 'iu_1666',
  'impression': '[REDACTED] airspace opacity in the left upper lung which may represent streaky atelectasis or resolving pneumonia'},
 {'uuid'

In [81]:
with open(SAMPLED_IMPRESSIONS_PATH, "w") as f:
    for rec in sampled_qa_pool:
        json.dump(rec, f)
        f.write("\n")

print(f"Saved {len(sampled_qa_pool)} sampled impressions to: {SAMPLED_IMPRESSIONS_PATH}")

Saved 520 sampled impressions to: /content/drive/MyDrive/multimodal-xray-agent/data/iu_xray/iu_sampled_for_qa.jsonl


## Step 3: Generating QA Pairs

In [7]:
# Static questions
QUESTION_1 = "What is the radiologic impression?"
QUESTION_2 = "Summarize the key thoracic findings."
PARAPHRASE_QUESTION_POOL = [
    "State the impression.",
    "Give a brief diagnostic conclusion.",
    "What are the radiographic findings?",
    "Provide a concise report summary."
]

In [8]:
# Load impressions
with open(SAMPLED_IMPRESSIONS_PATH, "r") as f:
    sampled_records = [json.loads(line) for line in f]

In [96]:
# Generate Q/A pairs
qa_records = []

for rec in tqdm(sampled_records, desc="Generating Q/A pairs"):
    uuid = rec["uuid"]
    impression = rec["impression"]

    # Template Q1 + Q2
    qa_records.append({"uuid": uuid, "question": QUESTION_1, "answer": impression})
    qa_records.append({"uuid": uuid, "question": QUESTION_2, "answer": impression})

    # GPT Q3
    paraphrase_q = random.choice(PARAPHRASE_QUESTION_POOL)
    paraphrased = gpt_paraphrase(impression)

    if paraphrased:
        qa_records.append({
            "uuid": uuid,
            "question": paraphrase_q,
            "answer": paraphrased
        })
    else:
        print(f"Dropped GPT-4o line for {uuid} — fallback triggered.")

Generating Q/A pairs: 100%|██████████| 520/520 [06:07<00:00,  1.42it/s]


In [110]:
qa_records[0:10]

[{'uuid': 'iu_5728',
  'question': 'What is the radiologic impression?',
  'answer': 'Right lower lobe pneumonia. Consider followup radiograph to document resolution'},
 {'uuid': 'iu_5728',
  'question': 'Summarize the key thoracic findings.',
  'answer': 'Right lower lobe pneumonia. Consider followup radiograph to document resolution'},
 {'uuid': 'iu_5728',
  'question': 'Give a brief diagnostic conclusion.',
  'answer': 'Right lower lobe pneumonia; recommend follow-up X-ray to confirm resolution.'},
 {'uuid': 'iu_3979',
  'question': 'What is the radiologic impression?',
  'answer': 'Small left retrocardiac opacity, may represent minimal atelectasis or small focus of airspace disease'},
 {'uuid': 'iu_3979',
  'question': 'Summarize the key thoracic findings.',
  'answer': 'Small left retrocardiac opacity, may represent minimal atelectasis or small focus of airspace disease'},
 {'uuid': 'iu_3979',
  'question': 'Give a brief diagnostic conclusion.',
  'answer': 'Small left retrocardia

In [100]:
# Save result
with open(OUTPUT_JSONL, "w") as f:
    for qa in qa_records:
        f.write(json.dumps(qa) + "\n")

print(f"\nSaved {len(qa_records)} total Q/A pairs to: {OUTPUT_JSONL}")


Saved 1560 total Q/A pairs to: /content/drive/MyDrive/multimodal-xray-agent/data/qapairs/qa_pairs_lora.jsonl


## Step 4: Diving the Training and Validation Sets

In [102]:
# Group QAs by UUID
random.seed(42)
by_uuid = {}
for qa in qa_records:
    by_uuid.setdefault(qa["uuid"], []).append(qa)

In [103]:
# Stratified UUID split
uuids = list(by_uuid)
random.shuffle(uuids)
cut = int(0.8 * len(uuids))
train = sum((by_uuid[u] for u in uuids[:cut]), [])
val   = sum((by_uuid[u] for u in uuids[cut:]), [])

In [105]:
dump(TRAIN_PATH, train)
dump(VAL_PATH, val)

print(f"train.jsonl = {len(train)} QAs")
print(f"val.jsonl   = {len(val)} QAs")

train.jsonl = 1248 QAs
val.jsonl   = 312 QAs
